# Desafio 2 - SQL

Uma universidade realiza uma competição em que alunos de diferentes escolas participam. As
categorias são Concurso Caribou, Debate, Feira de Ciências do Google, RoboCup e
Soletração. Os alunos participam de uma ou mais categorias e são pontuados de 0 a 100.

Escreva uma consulta que retorne os vencedores do 1º, 2º e 3º lugares em cada categoria.
Para determinar sua posição dentro de uma categoria, classifique as linhas primeiro por
pontuação decrescente e depois por nome da faculdade em ordem crescente. Ao relatar os
vencedores, classifique as linhas primeiro por categoria, em ordem crescente, e depois por
pontuação decrescente. As colunas a serem exibidas são categoria, student_id, nome,
nome_da_faculdade e pontuação.

`STUDENTS`

| id | name | college_name |
|----|------|--------------|
| 1352 | Howard Fields | University of California, Davis |
| 1801 | Tyrone Doyle | Santa Clara University |
| 2079 | Albert Burgess | Yale University |
| 2616 | Patrick Hum | Bryn Mawr College |
| 3920 | Diana Hum | Yale University |
| 4226 | Eugene Rogers | Dartmouth College |

`PARTICIPATIONS`

| participant_id | student_id | category | score |
|----------------|------------|----------|-------|
| 11853 | 5746 | Caribou Contest | 92 |
| 12114 | 2616 | Spelling Bee | 38 |
| 12267 | 2079 | Debate | 55 |
| 14036 | 2079 | Google Science Fair | 33 |
| 14149 | 1801 | Google Science Fair | 87 |
| 19674 | 6698 | RoboCup | 89 |
| 20679 | 5746 | RoboCup | 98 |
| 20718 | 8721 | Caribou Contest | 66 |
| 20847 | 2616 | Caribou Contest | 64 |

## Inicialização do banco de dados

In [1]:
import sqlite3

def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print(e)
    return conn

db_name = "competition.db"
conn = create_connection(db_name)
cursor = conn.cursor()

## Criação das tabelas

In [2]:
table_names = ["Students", "Participations"]

for table_name in table_names:
    try:
        cursor.execute(f"DROP TABLE IF EXISTS {table_name}")
    except sqlite3.OperationalError as e:
        print(f"Error dropping table '{table_name}': {e}")

In [3]:
cursor.execute('''
CREATE TABLE Students (
    id INTEGER PRIMARY KEY,
    name TEXT,
    college_name TEXT
)
''')

cursor.execute('''
CREATE TABLE Participations (
    participant_id INTEGER PRIMARY KEY,
    student_id INTEGER,
    category TEXT,
    score INTEGER,
    FOREIGN KEY (student_id) REFERENCES Students(id)
)
''')

conn.commit()


In [4]:
for table_name in table_names:
    print(f"Table: {table_name}")
    cursor.execute(f"PRAGMA table_info('{table_name}')")
    for row in cursor.fetchall():
        print(row)
    print("\n")

Table: Students
(0, 'id', 'INTEGER', 0, None, 1)
(1, 'name', 'TEXT', 0, None, 0)
(2, 'college_name', 'TEXT', 0, None, 0)


Table: Participations
(0, 'participant_id', 'INTEGER', 0, None, 1)
(1, 'student_id', 'INTEGER', 0, None, 0)
(2, 'category', 'TEXT', 0, None, 0)
(3, 'score', 'INTEGER', 0, None, 0)




## Inserção de dados

In [5]:
students_data = [
    (1352, 'Howard Fields', 'University of California, Davis'),
    (1801, 'Tyrone Doyle', 'Santa Clara University'),
    (2079, 'Albert Burgess', 'Yale University'),
    (2616, 'Patrick Hum', 'Bryn Mawr College'),
    (3920, 'Diana Hum', 'Yale University'),
    (4226, 'Eugene Rogers', 'Dartmouth College')
]

cursor.executemany('''
INSERT INTO Students (id, name, college_name)
VALUES (?, ?, ?)
''', students_data)

participations_data = [
    (11853, 5746, 'Caribou Contest', 92),
    (12114, 2616, 'Spelling Bee', 38),
    (12267, 2079, 'Debate', 55),
    (14036, 2079, 'Google Science Fair', 33),
    (14149, 1801, 'Google Science Fair', 87),
    (19674, 6698, 'RoboCup', 89),
    (20769, 5746, 'RoboCup', 98),
    (20718, 8721, 'Caribou Contest', 66),
    (20847, 2616, 'Caribou Contest', 64)
]

cursor.executemany('''
INSERT INTO Participations (participant_id, student_id, category, score)
VALUES (?, ?, ?, ?)
''', participations_data)

conn.commit()


In [6]:
for table_name in table_names:
    print(f"Table: {table_name}")
    cursor.execute(f"SELECT * FROM {table_name}")
    for row in cursor.fetchall():
        print(row)
    print("\n")

Table: Students
(1352, 'Howard Fields', 'University of California, Davis')
(1801, 'Tyrone Doyle', 'Santa Clara University')
(2079, 'Albert Burgess', 'Yale University')
(2616, 'Patrick Hum', 'Bryn Mawr College')
(3920, 'Diana Hum', 'Yale University')
(4226, 'Eugene Rogers', 'Dartmouth College')


Table: Participations
(11853, 5746, 'Caribou Contest', 92)
(12114, 2616, 'Spelling Bee', 38)
(12267, 2079, 'Debate', 55)
(14036, 2079, 'Google Science Fair', 33)
(14149, 1801, 'Google Science Fair', 87)
(19674, 6698, 'RoboCup', 89)
(20718, 8721, 'Caribou Contest', 66)
(20769, 5746, 'RoboCup', 98)
(20847, 2616, 'Caribou Contest', 64)




## Consulta

In [7]:
query = '''
WITH Ranked_Participations AS (
    SELECT
        p.category,
        p.student_id,
        s.name,
        s.college_name,
        p.score,
        ROW_NUMBER() OVER (PARTITION BY p.category ORDER BY p.score DESC, s.college_name ASC) AS rank
    FROM
        Participations p
    JOIN
        Students s ON p.student_id = s.id
)
SELECT
    category,
    student_id,
    name,
    college_name,
    score
FROM
    Ranked_Participations
WHERE
    rank <= 3
ORDER BY
    category ASC,
    score DESC
'''

cursor.execute(query)
results = cursor.fetchall()


### Exibição dos dados em formato de dataframe (com pandas) para facilitar visualização.

In [8]:
import pandas as pd
pd.set_option('display.expand_frame_repr', False)
df = pd.DataFrame(results, columns=['Category', 'Student ID', 'Name', 'College Name', 'Score'])
print(df)

              Category  Student ID            Name            College Name  Score
0      Caribou Contest        2616     Patrick Hum       Bryn Mawr College     64
1               Debate        2079  Albert Burgess         Yale University     55
2  Google Science Fair        1801    Tyrone Doyle  Santa Clara University     87
3  Google Science Fair        2079  Albert Burgess         Yale University     33
4         Spelling Bee        2616     Patrick Hum       Bryn Mawr College     38


In [9]:
conn.close()